# Supervised Learning: Text preprocessing

In [1]:
### load packages 
import numpy as np 
import pandas as pd
from autocorrect import spell 
import matplotlib
from nltk.stem import WordNetLemmatizer 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
from wordcloud import WordCloud, STOPWORDS
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
import re
import string
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stopwords.words('english')
import warnings
from nltk import word_tokenize, corpus
warnings.simplefilter("ignore")
import nltk
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
sns.set() # use seaborn plotting style
from nltk.tokenize import word_tokenize

In [6]:
### load data and merge them  
df=pd.read_csv("body_sentiment.csv")

# change column name 
df.rename({'clean_body': 'clean_text'}, axis=1, inplace=True)
#look at ddta
print(df.info()) 
sents_count=pd.DataFrame(df.groupby("Sentiment")["Sentiment"].count()).rename(columns={"Sentiment":"Entries"}).reset_index().style.set_caption("Sentiment overview")
sents_count

df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19244 entries, 0 to 19243
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Sentiment   19244 non-null  object
 1   score       19244 non-null  int64 
 2   comms_num   19244 non-null  int64 
 3   timestamp   19244 non-null  object
 4   clean_text  19018 non-null  object
dtypes: int64(2), object(3)
memory usage: 751.8+ KB
None


,Sentiment,score,comms_num,timestamp,clean_text
0,Negative,0,47,2021-01-28 21:30:35,the ceo of nasdaq pushed to halt trading to gi...
1,Negative,317,53,2021-01-28 21:26:27,hedgefund whales are spreading disinfo saying ...
2,Negative,405,178,2021-01-28 21:19:31,life isn fair my mother always told me that wh...
3,Positive,222,70,2021-01-28 21:18:25,i believe right now is one of those rare oppor...
4,Positive,0,16,2021-01-28 21:17:10,you guys are champs gme who would have thought...
...,...,...,...,...,...
19239,Neutral,130,13,2021-03-04 01:54:44,x200b
19240,Positive,105,72,2021-03-04 01:51:52,rkt as tech platform real estate doubled auto ...
19241,Positive,18,11,2021-03-04 01:41:49,vonage is the 2nd largest ucaas unified commun...
19242,Positive,296,46,2021-03-04 01:36:29,i don know if am an ape autist or retard ve be...


In [7]:
# dropping the columns we dont need
df.drop(columns=['score', 'comms_num', 'timestamp'], inplace=True)
df
print (list(df))
df['clean_text']=df['clean_text'].apply(str)


['Sentiment', 'clean_text']


In [9]:
def make_tokens(row):
    posts = row['clean_text']
    tokens = nltk.word_tokenize(posts)
    # taken only words (not punctuation)
    token_words = [w for w in tokens if w.isalpha()]
    return token_words

df['words'] = df.apply(make_tokens, axis=1)

#dropping the rows where the string is empty 
df=df[df['words'].map(lambda d: len(d)) > 0]


In [10]:
### Lemmatize

lemmatizer = WordNetLemmatizer() 

def Lemmatizer(row):
    my_list = row['words']
    lemma_list = [lemmatizer.lemmatize(word) for word in my_list]
    return (lemma_list)

df['lemmatized_words'] = df.apply(Lemmatizer, axis=1)



In [11]:
### removing stopwords 

stops = set(stopwords.words("english"))                  

def remove_stops(row):
    my_list = row['lemmatized_words']
    meaningful_words = [w for w in my_list if not w in stops]
    return (meaningful_words)

df['lemma_meaningful'] = df.apply(remove_stops, axis=1)


In [12]:
### make back into string 

def make_string(row):
    my_list = row['lemma_meaningful']
    joined_words = ( " ".join(my_list))
    return joined_words

df['processed'] = df.apply(make_string, axis=1)

In [13]:
#drop irrelavant columns 
df.drop(columns=['clean_text', 'words', 'lemmatized_words'], inplace=True)


In [14]:
## Create csv 

# create csv file 
df.to_csv('processed_body_data.csv',index=False )